In [26]:
from collections import defaultdict

# calculates the winners in a single transferable vote election. 
def stv(votes_dict, empty_seats):
    #initializes variables
    old_set = set()
    weightdict = defaultdict(lambda: 1.0)
    n_votes = len(votes_dict)
    n_round = 0
    
    # puts all contenders in a set
    cand_set = set()
    for vote in votes_dict.values():
        cand_set.update(vote)    

    # main loop
    winners = []
    while len(winners) < empty_seats:
        # update round number
        n_round+=1
        
        # tallies up all first places in each ranked vote
        cand_dict = defaultdict(int)
        for vote in votes_dict.keys():
            cand_dict[votes_dict[vote][0]] += 1 * weightdict[vote] 
        
        # see if there's any winners, if so add them to the list
        winner = False
        round_wins = []
        for i in cand_set:
            if cand_dict[i] > n_votes/(empty_seats + 1):
                winner = True
                round_wins.append(i)
                winners.append(i)
         
        # if there's any winners, print out who they are and remove them from rest of the race
        if winner:
            
            # update weights for the votes that voted for a winner
            for i in winners:
                if cand_dict[i] > n_votes/(empty_seats + 1):
                    for vote in votes_dict.keys():
                        if votes_dict[vote][0] == i:
                            weightdict[vote] *= (cand_dict[i] - n_votes/(empty_seats + 1))/cand_dict[i]
                
            # remove the winner from the votes and the candidate set    
            tmp = cand_set.copy()
            for i in cand_set:
                if i in winners:
                    tmp.remove(i)
                    for vote in votes_dict.keys():
                        try:
                            votes_dict[vote].remove(i)                                
                        except: 
                            pass
                        tmp2 = votes_dict.copy()
                        for key in votes_dict.keys():
                            if tmp2[key] == []:
                                del tmp2[key]
                        votes_dict = tmp2
            cand_set = tmp
            
            #print round number and winners this round
            print('round: {}'.format(n_round))
            print('winners: {}'.format(round_wins))
            
        # if there is no winner this round
        else:
            
            # finds the lowest amount of times any candidate came in first
            lowest = sorted([cand_dict[i] for i in cand_set])[0]

            # if the last n candidates are tied, 
            #   they'll all be removed from cand_set after the final iteration.
            # instead we want to return all of them which is why we keep track of the previous set too
            old_set = cand_set.copy()

            # iterate over all candidates and eliminate those 
            # that achieved the lowest score from both the candidate set 
            # and the actual votes
            elimns = []
            tmp = cand_set.copy()
            for i in cand_set:
                if cand_dict[i] == lowest:
                    tmp.remove(i)
                    elimns.append(i)
                    for vote in votes_dict.keys():
                        try:
                            votes_dict[vote].remove(i)                                
                        except: 
                            pass
                        tmp2 = votes_dict.copy()
                        for key in votes_dict.keys():
                            if tmp2[key] == []:
                                del tmp2[key]
                        votes_dict = tmp2
            cand_set = tmp
            
            #print round number and eliminations this round
            print('round: {}'.format(n_round))
            print('eliminations: {}'.format(elimns))
            
            # if there's an equal amount of empty seats and candidates not eliminated, they all win
            if len(cand_set) == empty_seats - len(winners):
                winners += list(cand_set)
                print('last person left: {}'.format(cand_set))
                
            # if there's less people left than empty seats, let all of them win, 
            # and add this round's eliminations to the win list as a tie
            elif len(cand_set) < empty_seats - len(winners):
                winners += list(cand_set)
                print('last tie left: {}'.format(old_set))
                winners.append(tuple(old_set.difference(cand_set)))
                break
 
    return winners

In [32]:
import re
import copy

empty_seats = 5
f = open("ir.txt", "r")

# extract ordered lists for each vote from logs of discord messages copied into a txt file.
sublist = []
votes_list = []
for line in f.read().split('\n'):
    if line[0].isnumeric():
        sublist.append(re.split(r'[\s\.,:]+', line)[1].lower())
    else: 
        votes_list.append(sublist)
        sublist = []
votes_list.append(sublist)

# correction for inconsistent naming due to nicknames/typoes
for vote in range(len(votes_list)):
    for person in range(len(votes_list[vote])):
        if votes_list[vote][person][:3] == 'tog':
            votes_list[vote][person] = 'thatotherguy'
        elif votes_list[vote][person][:3] == 'tri':
            votes_list[vote][person] = 'tricklejest'
        elif votes_list[vote][person][:3] == 'car':
            votes_list[vote][person] = 'carleah22'
        elif votes_list[vote][person][:3] == 'dag':
            votes_list[vote][person] = 'dagothur'
        elif votes_list[vote][person][:3] == 'noa':
            votes_list[vote][person] = 'noah610'
        elif votes_list[vote][person][:2] == 'al':
            votes_list[vote][person] = 'alastair'
        elif votes_list[vote][person][:2] == 'ta':
            votes_list[vote][person] = 'talist'
        elif votes_list[vote][person][:3] == 'sir':
            votes_list[vote][person] = 'sirplop'

# candidate dropped out and is removed
for i in votes_list:
    if 'alastair' in i:
        i.remove('alastair')
    
# convert list to dict with list index as keys
votes_dict = dict(zip(range(len(votes_list)), votes_list))
                  
# run stv print final results
print('\n final winners: {}'.format(stv(votes_dict, empty_seats)))

round: 1
winners: ['tricklejest', 'splashcat']
round: 2
winners: ['carleah22']
round: 3
eliminations: ['maniac', 'dagothur', 'sparked', 'robulus', 'noah610']
round: 4
eliminations: ['jaggerain']
round: 5
eliminations: ['nomble', 'sirplop']
round: 6
winners: ['thatotherguy']
round: 7
eliminations: ['fran']
round: 8
eliminations: ['karpinsky']
round: 9
eliminations: ['nedben']
round: 10
winners: ['dcccv']

 final winners: ['tricklejest', 'splashcat', 'carleah22', 'thatotherguy', 'dcccv']


In [84]:
{'dcccv': 2.827777777777778, 'karpinsky': 3.9055555555555563})


SyntaxError: invalid syntax (<ipython-input-84-4d4ffde0a460>, line 1)